In [1]:
import math
from toyclown import GeoUnit, ToyClown

In [2]:
hierarchy = [("Country", 1, 810), 
             ("State", 3, 270), 
             ("County", 3, 90), 
             ("Dist", 3, 30)]

eps_budget = 0.4998 # I picked a random number from one of Peter's runs
eps_splits = [0.25, 0.25, 0.25, 0.25] 

clown = ToyClown(hierarchy, eps_budget, eps_splits)
clown.show() 

Country
├── CountryState1
│   ├── CountryState1County1
│   │   ├── CountryState1County1Dist1
│   │   ├── CountryState1County1Dist2
│   │   └── CountryState1County1Dist3
│   ├── CountryState1County2
│   │   ├── CountryState1County2Dist1
│   │   ├── CountryState1County2Dist2
│   │   └── CountryState1County2Dist3
│   └── CountryState1County3
│       ├── CountryState1County3Dist1
│       ├── CountryState1County3Dist2
│       └── CountryState1County3Dist3
├── CountryState2
│   ├── CountryState2County1
│   │   ├── CountryState2County1Dist1
│   │   ├── CountryState2County1Dist2
│   │   └── CountryState2County1Dist3
│   ├── CountryState2County2
│   │   ├── CountryState2County2Dist1
│   │   ├── CountryState2County2Dist2
│   │   └── CountryState2County2Dist3
│   └── CountryState2County3
│       ├── CountryState2County3Dist1
│       ├── CountryState2County3Dist2
│       └── CountryState2County3Dist3
└── CountryState3
    ├── CountryState3County1
    │   ├── CountryState3County1Dist1
    │   ├── C

In [3]:
# This is where everything from Proposition 1 happens.
clown.noise_and_adjust()

In [4]:
# How to look at the node's attributes
clown.get_node("CountryState3").data.__dict__

{'name': 'CountryState3',
 'unnoised_pop': 270,
 'parent': 'Country',
 'identifier': 'CountryState3',
 'level': 1,
 'noised_pop': 276.66319391506784,
 'noise': 6.663193915067833,
 'noise_type': 'laplacian',
 'adjusted_pop': 279.43907465147197,
 'error': 9.43907465147197}

In [6]:
# A demonstration that the numbers add up within 0.00001
math.isclose(( clown.get_node("CountryState3County3Dist1").data.adjusted_pop \
             + clown.get_node("CountryState3County3Dist2").data.adjusted_pop \
             + clown.get_node("CountryState3County3Dist3").data.adjusted_pop ), 
             clown.get_node("CountryState3County3").data.adjusted_pop, 
             abs_tol=0.00001)

True